In [1]:
import numpy as np

class LogisticRegression:
    def __init__(self, learning_rate=0.01, momentum=0.9, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.epsilon = epsilon
        self.w = None
        self.v = None
        self.g2 = None
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def predict_proba(self, X):
        if self.w is None:
            raise ValueError("Model not trained yet")
        return self.sigmoid(X @ self.w)
    
    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)
    
    def fit(self, X, y, n_epochs=1000, batch_size=32):
        n_samples, n_features = X.shape
        self.w = np.zeros((n_features, 1))
        self.v = np.zeros((n_features, 1))
        self.g2 = np.zeros((n_features, 1))
        y = y.reshape(-1, 1)
        for epoch in range(n_epochs):
            shuffled_idx = np.random.permutation(n_samples)
            for batch_idx in range(0, n_samples, batch_size):
                batch_samples = shuffled_idx[batch_idx:batch_idx+batch_size]
                X_batch = X[batch_samples]
                y_batch = y[batch_samples]
                y_pred = self.predict_proba(X_batch)
                error = y_pred - y_batch
                gradient = X_batch.T @ error / batch_size
                self.g2 += gradient**2
                adaptive_lr = self.learning_rate / (self.epsilon + np.sqrt(self.g2))
                self.v = self.momentum * self.v - adaptive_lr * gradient
                self.w += self.v